
# Real-Time Query Reformulation a.k.a Query Correction using the UniversalDeepTransformer API

This notebook shows how to build a query reformulation model with ThirdAI's Universal Deep Transformer (UDT) model, our all-purpose solution for classification tasks on tabular datasets and query reformulation. In this demo, we will train and evaluate the model on a spelling correction dataset and show less than 5ms P99.9 inference latency.

You can immediately run a version of this notebook in your browser on Google Colab at the following link:

https://githubtocolab.com/ThirdAILabs/Demos/blob/main/QueryReformulation.ipynb

In [1]:
!pip3 install datasets==2.6.1 
!pip3 install thirdai --upgrade

Requirement already up-to-date: thirdai in /share/shubh/myenv/lib/python3.8/site-packages (0.5.13)


This notebook uses an activation key that will only work with this demo. If you want to try us out on your own dataset, you can obtain a free trial license at the following link: https://www.thirdai.com/try-bolt/

In [3]:
import thirdai
thirdai.licensing.activate("AWK9-WPMK-3NRE-AAAV-C39P-N9JV-43VC-CFUH")

## Dataset Download

We will use the demos module in the ThirdAI repo to download and pre-process a dataset from HuggingFace. The dataset we will use from HuggingFace is typically used for semantic sentence similarity. We will pre-process it by adding noise so that it is suitable for query reformulation. You can replace this step and the next with a UDT initialization that is specific for your dataset - as long as your train dataset consists of **CSV files with two string columns**: The first one should be incorrect queries and the second column will be their target reformulations.  The incorrect queries column can be empty strings

In [4]:
from thirdai.demos import prepare_query_reformulation_data

supervised_train_filename, unsupervised_train_filename, test_filename, inference_batch = prepare_query_reformulation_data()

/share/shubh/myenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration embedding-data--sentence-compression-d643585deb6e0073
Found cached dataset json (/home/shubh/.cache/huggingface/datasets/embedding-data___json/embedding-data--sentence-compression-d643585deb6e0073/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 260.26it/s]


## UDT Initialization

We can create a UDT model specific for query reformulation by specifying the name of the source column (column containing queries to be reformulated) and the name of the target column (correct reformulations) and a dataset size parameter. The size of the input dataset can be configured to be either "small" (size < 1M), "medium"(size < 10M) or "large" (size >= 10M). We configure different model parameters depending on the size of the input dataset. 

In [5]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    source_column="source_queries", target_column="target_queries", dataset_size="medium"
)

## Training

We can now train our model in just one line of code. You just have to specify the path to the training file. 
You can train the model in both **supervised** and **unsupervised** settings. When training the model in an unsupervised setting, only the target column is needed in the training file. 

You can pretrain your model by training in an unsupervised setting and directly evaluate it.
*unsupervised_train_filename* has only `target_queries` column.

In [6]:
# unsupervised training
model.train(filename=unsupervised_train_filename) 

loading data | source 'unsupervised_train_file.csv' | vectors 900000 | batches 90 | time 0s | complete

train | batches 90 | complete                                           ======================                        ] 51%



## Evaluation 

Evaluating the UDT model is also just one line of code. Since this UDT model is specific for query reformulation, you need to provide the number of suggested candidate queries that the UDT model generates. For instance, if you want to see the top 5 suggested query reformulations of the input query, set the `top_k` parameter to 5. Evaluating this model will also print out recall @k. If you also want the scores for the reformulated queries, you can specify `return_scores = True`

In [7]:
query_reformulations, scores = model.evaluate(filename=test_filename, top_k=5, return_scores = True)

loading data | source 'test_file.csv' | vectors 270000 | batches 27 | time 0s | complete

evaluate | batches 27 | complete                                           

Recall@5 = 0.840726


If you have supervised data, i.e., `source_queries`,`target_queries` pairs, then you can train the same model in a supervised setting. The APIs for training and evaluation remains the same. 
_supervised_train_filename_ has two columns namely `source_queries`, `target_queries`.

In [8]:
# supervised training
model.train(filename=supervised_train_filename)

loading data | source 'supervised_train_file.csv' | vectors 900000 | batches 90 | time 0s | complete

train | batches 90 | complete                                           



In [9]:
query_reformulations, = model.evaluate(filename=test_filename, top_k=5)

loading data | source 'test_file.csv' | vectors 270000 | batches 27 | time 0s | complete

evaluate | batches 27 | complete                =============] 100%                           

Recall@5 = 0.884885


## Saving and Loading

Saving and loading a trained UDT model to disk is also extremely straight forward. 

In [10]:
model_location = "query_reformulation.model"

# Saving
model.save(filename=model_location)

# Loading
model = bolt.UniversalDeepTransformer.load(model_location)

## Testing Predictions 

The evaluation method is great for testing, but it requires labels, which don't exist in a production environment. We also provide a predict method that can take a list of queries or a single query, which allows for easy integration into production pipelines. 

In [11]:
predictions, = model.predict_batch(queries=inference_batch, top_k=5)

In [12]:
model.predict(query="Health iiaclfsfo susei der air quality alert", top_k=5)

([['Health officials issue red air quality alert',
   'Air quality improves',
   'Ahs lifts air quality advisory',
   'Quality Health breaks ground',
   'Air quality officials issue health notice']],)